**אימפורטים של קיצורים לסיפריות נחוצות לעבודה עם מאגרי נתונים ויצירת פלוטים ויומן חודש**

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import calendar



data = pd.read_csv("reservationsN.csv")



print(data.head())


   cruise cancel   TTT       Y     M     W  DOM  weekendnights  weeknights  \
0  Deluxe     no  69.0  2015.0  July  27.0  2.0            2.0         4.0   
1  Deluxe    yes  45.0  2015.0  July  27.0  2.0            1.0         3.0   
2  Deluxe    yes  40.0  2015.0  July  27.0  2.0            1.0         3.0   
3  Deluxe     no  15.0  2015.0  July  27.0  2.0            1.0         3.0   
4  Deluxe     no  36.0  2015.0  July  27.0  2.0            1.0         3.0   

   ADLT  ...     deposit  agentNr CMPY waiting         CT DailyAVG  ExtraLock  \
0   2.0  ...  No Deposit    175.0  NaN     0.0  Transient     65.5        0.0   
1   3.0  ...  No Deposit    241.0  NaN     0.0  Transient    108.8        0.0   
2   3.0  ...  No Deposit    241.0  NaN     0.0  Transient    108.8        0.0   
3   2.0  ...  No Deposit    240.0  NaN     0.0  Transient     98.0        0.0   
4   3.0  ...  No Deposit    241.0  NaN     0.0  Transient    108.8        0.0   

   Special       stat      statD  
0      0.

**פונקציה המקבלת את החודש השנה וסוג השייט הרצוי מהמהשתמש**

In [6]:

def get_user_input():
    while True:
        try:
            month = int(input("Enter the month (1-12): "))
            if month < 1 or month > 12:
                raise ValueError("Month must be between 1 and 12")
            year = int(input("Enter the year: "))
            cruise_type = input("Enter the cruise type: ")
            return month, year, cruise_type
        except ValueError as e:
            print(f"Error: {e}. Please enter a valid month (1-12).")


**פונקציה שמסננת את הדאטה שיהיה טיידי**

In [7]:


def filter_data(data, month, year, cruise_type):
    data['statD'] = pd.to_datetime(data['statD'], errors='coerce')
    filtered_data = data[(data['statD'].dt.month == month) &
                         (data['statD'].dt.year == year) &
                         (data['cruise'] == cruise_type)]
    return filtered_data


**בקטע הזה קיימות שלוש פונקציות עיקריות אחת שיוצרת את מראה היומן השניה מציירת את הימים על הגרף והשלישית מוסיפה את הצבעים המבוקשים**

In [10]:

def create_calendar_plot(filtered_data, month, year):
    days_in_month = calendar.monthrange(year, month)[1]
    occupancy_data = np.zeros(days_in_month)
    
    for day in range(1, days_in_month + 1):
        day_data = filtered_data[filtered_data['statD'].dt.day == day]
        if not day_data.empty:
            occupancy_data[day - 1] = day_data['TTT'].values[0]  # assuming 'TTT' is the occupancy column
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(f"Occupancy for {calendar.month_name[month]} {year}", fontsize=16)
    
    cmap = plt.cm.get_cmap('YlGn')
    norm = mcolors.Normalize(vmin=0, vmax=100)
    
    start_day_of_week = calendar.monthrange(year, month)[0]
    
    
  
    for day in range(1, days_in_month + 1):
        row = (start_day_of_week + day - 1) // 7 + 1
        col = (start_day_of_week + day - 1) % 7
        color = cmap(norm(occupancy_data[day - 1]))
        ax.add_patch(plt.Rectangle((col, -row), 1, 1, edgecolor='black', facecolor=color))
        ax.text(col + 0.5, -row + 0.5, str(day),
                horizontalalignment='center', verticalalignment='center')
    
    
    ax.set_xlim(0, 7)
    ax.set_ylim(-((start_day_of_week + days_in_month + 6) // 7), 0)
    ax.axis('off')
    
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, orientation='vertical', ax=ax, pad=0.02)
    cbar.set_label('Occupancy (%)')
    
    plt.show()

**פה אנו מקבלים את המידע מהמשתמש מסננים את הדאטה ויוצרים את הפלוט במימושים של הפונקציות שיצרנו**

In [ ]:

month, year, cruise_type = get_user_input()


filtered_data = filter_data(data, month, year, cruise_type)


create_calendar_plot(filtered_data, month, year)